In [ ]:
# ==========================================================
# JAFFE + PCA + SVM - Emotion Recognition Notebook [RBF optim]
# ==========================================================
# This notebook contains the full project code with explanations,
# and it can be run interactively in Jupyter with color-coded syntax.

# Import libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# -----------------------------
# Load JAFFE dataset
# -----------------------------
data_dir = "../data"  # Adjust path to your JAFFE dataset
images = [f for f in os.listdir(data_dir) if f.endswith(".tiff")]

# -----------------------------
# Prepare data and labels
# -----------------------------
X = []  # Image data
y = []  # Emotion labels

emotion_map = {
    "HA": "Happy",
    "SU": "Surprise",
    "AN": "Angry",
    "NE": "Neutral",
    "DI": "Disgust",
    "FE": "Fear",
    "SA": "Sad"
}

for f in images:
    img = cv2.imread(os.path.join(data_dir, f), cv2.IMREAD_GRAYSCALE)
    X.append(img.flatten())
    emotion_code = f.split(".")[1][:2]
    y.append(emotion_map.get(emotion_code, "Unknown"))

X = np.array(X)
y = np.array(y)

# -----------------------------
# Split dataset
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=42, stratify=y
)

# -----------------------------
# PCA
# -----------------------------
pca = PCA(n_components=70)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# -----------------------------
# Train SVM
# -----------------------------
svm = SVC(kernel='rbf', C=10, gamma='scale')
svm.fit(X_train_pca, y_train)

# -----------------------------
# Evaluate
# -----------------------------
y_pred = svm.predict(X_test_pca)
print(f"Test Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(classification_report(y_test, y_pred))

# -----------------------------
# Confusion matrix
# -----------------------------
cm = confusion_matrix(y_test, y_pred, labels=list(emotion_map.values()))
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Purples', xticklabels=emotion_map.values(), yticklabels=emotion_map.values())
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix - RBF Kernel SVM')
plt.show()

# -----------------------------
# Visualize 9 random test images
# -----------------------------
num_samples = 9
plt.figure(figsize=(12, 12))
indices = np.random.choice(len(X_test), num_samples, replace=False)

for i, idx in enumerate(indices):
    img = X_test[idx].reshape(256, 256)
    predicted = y_pred[idx]
    true = y_test[idx]

    plt.subplot(3, 3, i + 1)
    plt.imshow(img, cmap='gray')
    plt.title(f"P: {predicted}\nT: {true}", fontsize=10,
              color='green' if predicted == true else 'red')
    plt.axis('off')

plt.suptitle("Predicted (P) vs True (T) Emotions", fontsize=14)
plt.tight_layout()
plt.show()

# -----------------------------
# PCA 2D scatter plot
# -----------------------------
if X_train_pca.shape[1] >= 2:
    plt.figure(figsize=(8, 6))
    emotion_list = ['Happy', 'Sad', 'Angry', 'Neutral', 'Surprise', 'Disgust', 'Fear']
    color_map = {emotion: i for i, emotion in enumerate(emotion_list)}
    colors = [color_map[label] for label in y_train]

    scatter = plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], c=colors, cmap='tab10', alpha=0.7)
    plt.xlabel('PCA Component 1')
    plt.ylabel('PCA Component 2')
    plt.title('PCA projection of training images colored by emotion')

    cbar = plt.colorbar(scatter, ticks=range(len(emotion_list)))
    cbar.ax.set_yticklabels(emotion_list)
    plt.clim(-0.5, len(emotion_list) - 0.5)
    plt.show()


In [ ]:
# ==========================================================
# JAFFE + PCA + SVM (Polynomial Kernel) - Optimized [Poly]
# ==========================================================
import os
import cv2
import numpy as np
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Load JAFFE dataset
data_dir = "../data"  # Ajustează calea către dataset
images = [f for f in os.listdir(data_dir) if f.endswith(".tiff")]

X = []
y = []

# Map emoții
emotion_map = {
    "HA": "Happy",
    "SU": "Surprise",
    "AN": "Angry",
    "NE": "Neutral",
    "DI": "Disgust",
    "FE": "Fear",
    "SA": "Sad"
}

# Preprocesare imagini și etichete
for f in images:
    img = cv2.imread(os.path.join(data_dir, f), cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (80, 80))
    img = cv2.equalizeHist(img)
    img = img / 255.0

    X.append(img.flatten())
    emotion_code = f.split(".")[1][:2]
    y.append(emotion_map.get(emotion_code, "Unknown"))

X = np.array(X)
y = np.array(y)

# Împărțire train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.37, random_state=42, stratify=y
)

# PCA (păstrează 99.9% din varianță)
pca = PCA(0.999, whiten=True)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)
print("PCA components used:", pca.n_components_)

# Standard scaling
scaler = StandardScaler()
X_train_pca = scaler.fit_transform(X_train_pca)
X_test_pca = scaler.transform(X_test_pca)

# Polynomial SVM + GridSearchCV
param_grid = {
    'C': [1, 5, 10, 20, 50],
    'degree': [2, 3, 4],
    'gamma': ['scale', 0.0005, 0.001, 0.005]
}

grid = GridSearchCV(
    SVC(kernel='poly', class_weight='balanced'),
    param_grid,
    cv=5,
    n_jobs=-1
)

grid.fit(X_train_pca, y_train)
svm_poly = grid.best_estimator_
print("Best Polynomial SVM params:", grid.best_params_)

# Evaluare performanță
y_pred = svm_poly.predict(X_test_pca)
print("Test accuracy (Polynomial Kernel - Optimized):", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Matricea de confuzie
cm = confusion_matrix(y_test, y_pred, labels=list(emotion_map.values()))
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Purples', xticklabels=emotion_map.values(), yticklabels=emotion_map.values())
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix - Polynomial Kernel SVM')
plt.show()


In [ ]:
# ==========================================================
# JAFFE + PCA + SVM - Linear Kernel [linear]
# ==========================================================
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# -----------------------------
# Load JAFFE dataset
# -----------------------------
data_dir = "../data"
images = [f for f in os.listdir(data_dir) if f.endswith(".tiff")]

# -----------------------------
# Prepare data and labels
# -----------------------------
X, y = [], []

emotion_map = {
    "HA": "Happy",
    "SU": "Surprise",
    "AN": "Angry",
    "NE": "Neutral",
    "DI": "Disgust",
    "FE": "Fear",
    "SA": "Sad"
}

for f in images:
    img = cv2.imread(os.path.join(data_dir, f), cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (64, 64))
    img = cv2.equalizeHist(img)
    img = img / 255.0
    X.append(img.flatten())
    emotion_code = f.split(".")[1][:2]
    y.append(emotion_map.get(emotion_code, "Unknown"))

X = np.array(X)
y = np.array(y)

# -----------------------------
# Train/test split
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# -----------------------------
# PCA for dimensionality reduction
# -----------------------------
pca = PCA(n_components=70, whiten=True)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# -----------------------------
#  Scale PCA features
# -----------------------------
scaler = StandardScaler()
X_train_pca = scaler.fit_transform(X_train_pca)
X_test_pca = scaler.transform(X_test_pca)

# -----------------------------
# Train Linear SVM
# -----------------------------
svm_linear = SVC(kernel='linear', C=10, class_weight='balanced')
svm_linear.fit(X_train_pca, y_train)

# -----------------------------
# Evaluate
# -----------------------------
y_pred = svm_linear.predict(X_test_pca)
print("Linear SVM Test Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# -----------------------------
# Confusion matrix
# -----------------------------
cm = confusion_matrix(y_test, y_pred, labels=list(emotion_map.values()))
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Purples', xticklabels=emotion_map.values(), yticklabels=emotion_map.values())
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix - Linear Kernel SVM')
plt.show()
